In [1]:

####################################################################################################
####################################################################################################

import sys
import glob
import os.path

import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

from IPython.display import display
from IPython.display import Image
from PIL import Image

import keras
from keras import backend as K
from keras import layers
from keras import regularizers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

sys.path.append(os.path.abspath("../../"))
sys.path.append(os.path.abspath("../../pax_utils"))
from s1s2_utils import *

from model_xy_s2waveforms_lstm import *
from get_dnn_data import *



Using TensorFlow backend.


In [4]:

####################################################################################################
# Training Data
####################################################################################################

nEventsTrain = 1000
nEventsTrain = 1

input_dir  = "../../pax_merge/merged/"
input_file = 'merged_all_0k_test.pkl'
input_path = input_dir + input_file

s2cols = getS2integralsDataFrameColumns()
#print(s2cols)

cols = [
    'event_number',
    'event_s2_count',
    'event_s2_length', 
    'event_s2_left',
    'event_s2_right', 
    'intr_count',
    'intr_corrected_s2_area',
    'intr_s2_electrons',
    'x',
    'y',
    's2_area_000',
    's2_area_001',
    's2_area_002',
    's2_area_003',
    
]

df_events, s2_window_max = getEventsDataFrame(input_path)
df_events                = df_events[0:nEventsTrain][:].copy(deep=True)
df_s2integrals           = df_events[s2cols]

df_events                = df_events[cols]
#df_events                = df_events[0:10][:]


print(df_events.shape)
display(df_events[0:5][:])

print()
print("Event Max S2 Window size: " + str(s2_window_max))
print()
        


(1, 14)


,event_number,event_s2_count,event_s2_length,event_s2_left,event_s2_right,intr_count,intr_corrected_s2_area,intr_s2_electrons,x,y,s2_area_000,s2_area_001,s2_area_002,s2_area_003
1,1,6,463,105426,105888,1,2102.227029,98.696105,2.626434,-17.960824,1.715228,0.0,0.0,0.0



Event Max S2 Window size: 1448



In [5]:

####################################################################################################
# Training Data
####################################################################################################

train_data, train_truth  = get_data(df_events, s2_window_max)

print()
print("Input data shape:       " + str(train_data.shape ))
print("Truth data shape:       " + str(train_truth.shape))
print()


0 empty events

Input data shape:       (1, 183896)
Truth data shape:       (1, 2)



In [6]:

####################################################################################################
####################################################################################################

n_events     = nEventsTrain
n_channels   = 127
n_outputs    = 2
n_timesteps  = s2_window_max
go_backwards = False
unroll       = False


####################################################################################################
# Training Data
####################################################################################################

#train_data  = np.random.random( (n_events, n_timesteps, n_channels) )
#train_truth = np.random.random( (n_events, n_outputs)               )

print()
print("Channels:         " + str(n_channels))
print("Timesteps:        " + str(n_timesteps))
print("Outputs:          " + str(n_outputs) )
print("Input data shape: " + str(train_data.shape ))
print("Truth data shape: " + str(train_truth.shape))
print()




Channels:         127
Timesteps:        1448
Outputs:          2
Input data shape: (1, 183896)
Truth data shape: (1, 2)



In [ ]:

####################################################################################################
# To Do: understand stateful, return_state
####################################################################################################

model, name = dnnModel(n_channels, n_timesteps, n_outputs, 'elu', 0.00005, go_backwards, unroll)


######################################################################################
######################################################################################

name_png = 'models/' + name + ".png"

print()
display(Image.open(name_png))
print()



In [7]:

######################################################################################
######################################################################################

model.fit(
    train_data,
    train_truth,
    batch_size=64,
    epochs=1,
    verbose=True
)

print("Training Complete.")


Epoch 1/1
1000/1000 [==============================] - 392s 392ms/step - loss: 390.2798
Training Complete.


In [8]:

######################################################################################
# Save Model
######################################################################################

folder  = "models/"    
name_h5 = folder + name + ".h5"

model.save(name_h5, overwrite=True)

print(name_h5)


models/model_xy_s2waveforms_lstm_elu.h5
